# Decorators and Closures

In [10]:
import time
import functools
from dis import dis

## Decorators 101

In [11]:
def deco(func):
    def inner():
        print('running inner()')
    return inner


@deco
def target():
    print('running target()')


target()        # invoking the decorated `target` actually runs `inner`
print(target)


running inner()
<function deco.<locals>.inner at 0x000001D2BE364430>


function decorators are executed as soon as the model is imported

In [12]:
registry = []  


def register(func):  
    print(f'running register({func})') 
    registry.append(func) 
    return func  


@register  
def f1():
    print('running f1()')


@register
def f2():
    print('running f2()')


def f3():  
    print('running f3()')


print('running main()')
print('registry ->', registry)
f1()
f2()
f3()


running register(<function f1 at 0x000001D2BE2B7AC0>)
running register(<function f2 at 0x000001D2BE2B7760>)
running main()
registry -> [<function f1 at 0x000001D2BE2B7AC0>, <function f2 at 0x000001D2BE2B7760>]
running f1()
running f2()
running f3()


In [13]:
b = 6


def f(a):
    global b    # if not this statement, `print(b)` will throw error
    print(a)
    print(b)


f(3)
dis(f)


3
6
  6           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  7           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


## Closures

In [14]:
def make_averager():
    series = []                 # closure

    def averager(value):
        series.append(value)    # free variable
        total = sum(series)
        return total / len(series)

    return averager


When invoked, `make_averager` returns an `averager` function object. 
Each time an `averager` is called, it appends the passed argument to the series. 
`avg` is a inner function, `averager`. 

In [15]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12), "\n")

print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

10.0
10.5
11.0 

('value', 'total')
('series',)
(<cell at 0x000001D2BE1FBE80: list object at 0x000001D2BE24C3C0>,)
[10, 11, 12]


## The `nonlocal` Declaration

In [16]:
def outer_function():
    x = 10

    def inner_function():
        nonlocal x
        x += 5
        print(x)

    inner_function()


outer_function()  # output 15

15


In [17]:
def clock(fun):
    def clocked(*args):
        t0 = time.perf_counter()
        result = fun(*args)
        elapsed = time.perf_counter() - t0
        name = fun.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked


@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6 !=', factorial(6))


**************************************** Calling snooze(.123)
[0.13056040s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000050s] factorial(1) -> 1
[0.00000880s] factorial(2) -> 2
[0.00001360s] factorial(3) -> 6
[0.00001790s] factorial(4) -> 24
[0.00002220s] factorial(5) -> 120
[0.00002710s] factorial(6) -> 720
6 != 720


## Decorators in the Standard Library
### @cache

In [18]:
@functools.cache
@clock              # `@cache` is applied on the function returned by `@clock`
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


fibonacci(6)

[0.00000060s] fibonacci(0) -> 0
[0.00000080s] fibonacci(1) -> 1
[0.00032490s] fibonacci(2) -> 1
[0.00000070s] fibonacci(3) -> 2
[0.00034110s] fibonacci(4) -> 3
[0.00000050s] fibonacci(5) -> 5
[0.00035280s] fibonacci(6) -> 8



### @lru_cache

In [24]:
functools.lru_cache(maxsize=2**5, typed=False)
@clock              
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


fibonacci(6)

[0.00000090s] fibonacci(0) -> 0
[0.00000080s] fibonacci(1) -> 1
[0.00048810s] fibonacci(2) -> 1
[0.00000030s] fibonacci(1) -> 1
[0.00000040s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00001060s] fibonacci(2) -> 1
[0.00002060s] fibonacci(3) -> 2
[0.00051970s] fibonacci(4) -> 3
[0.00000030s] fibonacci(1) -> 1
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00000990s] fibonacci(2) -> 1
[0.00002050s] fibonacci(3) -> 2
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001000s] fibonacci(2) -> 1
[0.00000030s] fibonacci(1) -> 1
[0.00000090s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001200s] fibonacci(2) -> 1
[0.00002260s] fibonacci(3) -> 2
[0.00004350s] fibonacci(4) -> 3
[0.00007420s] fibonacci(5) -> 5
[0.00060530s] fibonacci(6) -> 8



In [28]:
import html
import decimal


def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'


print(htmlize({1, 2, 3}))
print(htmlize(['alpha', 66, {3, 2, 1}]))
print(htmlize(abs))
print(htmlize(42))
print(htmlize(True))
print(htmlize(2 / 3))
print(htmlize(decimal.Decimal('0.02380953')))


<pre>{1, 2, 3}</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>42</pre>
<pre>True</pre>
<pre>0.6666666666666666</pre>
<pre>Decimal(&#x27;0.02380953&#x27;)</pre>


### @singledispatch
**Overloading**

In [29]:
from functools import singledispatch
from collections import abc
import fractions
import numbers

@singledispatch  
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register  
def _(text: str) -> str:  
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register  
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register  
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register  
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction) 
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal) 
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'


print(htmlize({1, 2, 3}))
print(htmlize(['alpha', 66, {3, 2, 1}]))
print(htmlize(abs))
print(htmlize(42))
print(htmlize(True))
print(htmlize(2 / 3))
print(htmlize(decimal.Decimal('0.02380953')))

<pre>{1, 2, 3}</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>
<pre>&lt;built-in function abs&gt;</pre>
<pre>42 (0x2a)</pre>
<pre>True</pre>
<pre>0.6666666666666666 (2/3)</pre>
<pre>0.02380953 (1/42)</pre>


## Parameterized Decorators
Now `registry` is a `set`, so adding and removing functions is faster. 

In [10]:
registry = set() 


def register(active=True):  
    def decorator(func):
        print(f'running register'
              f'(active={active})->decorate({func})') 
        if active:
            registry.add(func) 
        else:
            registry.discard(func)
        return func  
    return decorator


@register(active=False) 
def f1():
    print('running f1()')


@register()
def f2():
    print('running f2()')


def f3():  
    print('running f3()')


f1()
f2()
f3()

print(registry, "\n")
register()(f3)

print()
register(active=False)(f2)


running register(active=False)->decorate(<function f1 at 0x000001F8A03B0430>)
running register(active=True)->decorate(<function f2 at 0x000001F8A0310F70>)
running f1()
running f2()
running f3()
{<function f2 at 0x000001F8A0310F70>} 

running register(active=True)->decorate(<function f3 at 0x000001F8A0313010>)

running register(active=False)->decorate(<function f2 at 0x000001F8A0310F70>)


<function __main__.f2()>

Using `**locals()` below allows any local variable of `clocked` to be referenced in the `fmt`. 

In [9]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'


def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


@clock()        
def snooze1(second):
    time.sleep(second)


@clock('{name}: {elapsed}s')
def snooze2(second):
    time.sleep(second)

for i in range(3):
    snooze1(.123)
for i in range(3):
    snooze2(.123)

[0.12544630s] snooze1(0.123) -> None
[0.12433910s] snooze1(0.123) -> None
[0.12558620s] snooze1(0.123) -> None
snooze2: 0.12385730000005424s
snooze2: 0.12471880000020974s
snooze2: 0.12444709999999759s


### A Class-Based Decorator

In [12]:
import time


DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'


class clock_1:

    def __init__(self, fmt = DEFAULT_FMT):
        self.fmt = fmt
    
    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked


@clock_1()        
def snooze1(second):
    time.sleep(second)


@clock_1('{name}: {elapsed}s')
def snooze2(second):
    time.sleep(second)

for i in range(3):
    snooze1(.123)
for i in range(3):
    snooze2(.123)

[0.12692720s] snooze1(0.123) -> None
[0.13574420s] snooze1(0.123) -> None
[0.12297400s] snooze1(0.123) -> None
snooze2: 0.12984659999983705s
snooze2: 0.12850049999997282s
snooze2: 0.12944830000014917s
